<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Oliver", "transactions": [{"transaction-id": 123, "amount": 344}, {"transaction-id": 179, "amount": 355}, {"transaction-id": 317, "amount": 310}, {"transaction-id": 412, "amount": 335}, {"transaction-id": 704, "amount": 340}, {"transaction-id": 769, "amount": 348}, {"transaction-id": 818, "amount": 348}, {"transaction-id": 1000, "amount": 341}, {"transaction-id": 1256, "amount": 350}, {"transaction-id": 1307, "amount": 367}, {"transaction-id": 1494, "amount": 362}, {"transaction-id": 1557, "amount": 326}, {"transaction-id": 1582, "amount": 337}, {"transaction-id": 1621, "amount": 371}, {"transaction-id": 1645, "amount": 311}, {"transaction-id": 1908, "amount": 367}, {"transaction-id": 2125, "amount": 287}, {"transaction-id": 2183, "amount": 339}, {"transaction-id": 2353, "amount": 353}, {"transaction-id": 2379, "amount": 355}, {"transaction-id": 2424, "amount": 343}, {"transaction-id": 2577, "amount": 297}, {"transaction-id": 2583, "amount": 325}, {"transaction-id"

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Oliver", "transactions": [{"transaction-id": 123, "amount": 344}, {"transaction-id": 179, "amount": 355}, {"transaction-id": 317, "amount": 310}, {"transaction-id": 412, "amount": 335}, {"transaction-id": 704, "amount": 340}, {"transaction-id": 769, "amount": 348}, {"transaction-id": 818, "amount": 348}, {"transaction-id": 1000, "amount": 341}, {"transaction-id": 1256, "amount": 350}, {"transaction-id": 1307, "amount": 367}, {"transaction-id": 1494, "amount": 362}, {"transaction-id": 1557, "amount": 326}, {"transaction-id": 1582, "amount": 337}, {"transaction-id": 1621, "amount": 371}, {"transaction-id": 1645, "amount": 311}, {"transaction-id": 1908, "amount": 367}, {"transaction-id": 2125, "amount": 287}, {"transaction-id": 2183, "amount": 339}, {"transaction-id": 2353, "amount": 353}, {"transaction-id": 2379, "amount": 355}, {"transaction-id": 2424, "amount": 343}, {"transaction-id": 2577, "amount": 297}, {"transaction-id": 2583, "amount": 325}, {"transaction-id"

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Oliver',
  'transactions': [{'transaction-id': 123, 'amount': 344},
   {'transaction-id': 179, 'amount': 355},
   {'transaction-id': 317, 'amount': 310},
   {'transaction-id': 412, 'amount': 335},
   {'transaction-id': 704, 'amount': 340},
   {'transaction-id': 769, 'amount': 348},
   {'transaction-id': 818, 'amount': 348},
   {'transaction-id': 1000, 'amount': 341},
   {'transaction-id': 1256, 'amount': 350},
   {'transaction-id': 1307, 'amount': 367},
   {'transaction-id': 1494, 'amount': 362},
   {'transaction-id': 1557, 'amount': 326},
   {'transaction-id': 1582, 'amount': 337},
   {'transaction-id': 1621, 'amount': 371},
   {'transaction-id': 1645, 'amount': 311},
   {'transaction-id': 1908, 'amount': 367},
   {'transaction-id': 2125, 'amount': 287},
   {'transaction-id': 2183, 'amount': 339},
   {'transaction-id': 2353, 'amount': 353},
   {'transaction-id': 2379, 'amount': 355},
   {'transaction-id': 2424, 'amount': 343},
   {'transaction-id': 2577, 'amount'

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 7,
  'name': 'Alice',
  'transactions': [{'transaction-id': 1853, 'amount': 611},
   {'transaction-id': 2511, 'amount': 1269},
   {'transaction-id': 2537, 'amount': 1103},
   {'transaction-id': 2669, 'amount': 1161},
   {'transaction-id': 2927, 'amount': 888},
   {'transaction-id': 5060, 'amount': 979},
   {'transaction-id': 5168, 'amount': 973},
   {'transaction-id': 5393, 'amount': 861},
   {'transaction-id': 5767, 'amount': 783},
   {'transaction-id': 6269, 'amount': 1097},
   {'transaction-id': 6462, 'amount': 845},
   {'transaction-id': 6503, 'amount': 656},
   {'transaction-id': 6703, 'amount': 946},
   {'transaction-id': 6736, 'amount': 922},
   {'transaction-id': 7159, 'amount': 1205},
   {'transaction-id': 7288, 'amount': 657},
   {'transaction-id': 8230, 'amount': 929},
   {'transaction-id': 9189, 'amount': 1115}]},
 {'id': 8,
  'name': 'Alice',
  'transactions': [{'transaction-id': 440, 'amount': 28},
   {'transaction-id': 864, 'amount': 34},
   {'transaction-id': 15

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 18},
 {'name': 'Alice', 'count': 26},
 {'name': 'Alice', 'count': 25},
 {'name': 'Alice', 'count': 21},
 {'name': 'Alice', 'count': 33})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(18, 26, 25, 21, 33)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

30.63453237410072

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 1853, 'amount': 611},
  {'transaction-id': 2511, 'amount': 1269},
  {'transaction-id': 2537, 'amount': 1103},
  {'transaction-id': 2669, 'amount': 1161},
  {'transaction-id': 2927, 'amount': 888},
  {'transaction-id': 5060, 'amount': 979},
  {'transaction-id': 5168, 'amount': 973},
  {'transaction-id': 5393, 'amount': 861},
  {'transaction-id': 5767, 'amount': 783},
  {'transaction-id': 6269, 'amount': 1097},
  {'transaction-id': 6462, 'amount': 845},
  {'transaction-id': 6503, 'amount': 656},
  {'transaction-id': 6703, 'amount': 946},
  {'transaction-id': 6736, 'amount': 922},
  {'transaction-id': 7159, 'amount': 1205},
  {'transaction-id': 7288, 'amount': 657},
  {'transaction-id': 8230, 'amount': 929},
  {'transaction-id': 9189, 'amount': 1115}],
 [{'transaction-id': 440, 'amount': 28},
  {'transaction-id': 864, 'amount': 34},
  {'transaction-id': 1512, 'amount': 34},
  {'transaction-id': 1683, 'amount': 28},
  {'transaction-id': 2109, 'amount': 22},
  {'transac

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 1853, 'amount': 611},
 {'transaction-id': 2511, 'amount': 1269},
 {'transaction-id': 2537, 'amount': 1103})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(611, 1269, 1103)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

1708.4988023108356

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 166), ('Alice', 168), ('Alice', 180), ('Alice', 181), ('Bob', 80), ('Bob', 81), ('Bob', 87), ('Bob', 90), ('Charlie', 117), ('Charlie', 117), ('Charlie', 128), ('Charlie', 128), ('Dan', 87), ('Dan', 92), ('Dan', 96), ('Dan', 99), ('Edith', 96), ('Edith', 96), ('Edith', 104), ('Edith', 104), ('Frank', 126), ('Frank', 130), ('Frank', 140), ('Frank', 140), ('George', 132), ('George', 132), ('George', 142), ('George', 143), ('Hannah', 72), ('Hannah', 72), ('Hannah', 77), ('Hannah', 77), ('Ingrid', 105), ('Ingrid', 108), ('Ingrid', 116), ('Ingrid', 117), ('Jerry', 60), ('Jerry', 60), ('Jerry', 65), ('Jerry', 65), ('Kevin', 95), ('Kevin', 96), ('Kevin', 103), ('Kevin', 104), ('Laura', 127), ('Laura', 129), ('Laura', 140), ('Laura', 140), ('Michael', 108), ('Michael', 108), ('Michael', 117), ('Michael', 117), ('Norbert', 83), ('Norbert', 83), ('Norbert', 88), ('Norbert', 88), ('Oliver', 88), ('Oliver', 90), ('Oliver', 94), ('Oliver', 98), ('Patricia', 127), ('Patricia', 128), ('Pat

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 695), ('Bob', 338), ('Charlie', 490), ('Dan', 374), ('Edith', 400), ('Frank', 536), ('George', 549), ('Hannah', 298), ('Ingrid', 446), ('Jerry', 250), ('Kevin', 398), ('Laura', 536), ('Michael', 450), ('Norbert', 342), ('Oliver', 370), ('Patricia', 530), ('Quinn', 485), ('Ray', 676), ('Sarah', 200), ('Tim', 443), ('Ursula', 640), ('Victor', 398), ('Wendy', 614), ('Xavier', 499), ('Yvonne', 550), ('Zelda', 609)]
CPU times: user 113 ms, sys: 29.2 ms, total: 142 ms
Wall time: 466 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Oliver,"[{'transaction-id': 123, 'amount': 344}, {'tra..."
1,1,Patricia,"[{'transaction-id': 208, 'amount': 2804}, {'tr..."
2,2,Quinn,"[{'transaction-id': 78, 'amount': 627}, {'tran..."
3,4,Xavier,"[{'transaction-id': 92, 'amount': 2914}, {'tra..."
4,5,Ursula,"[{'transaction-id': 9, 'amount': 2541}, {'tran..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 237 ms, sys: 6.08 ms, total: 243 ms
Wall time: 1.36 s


name
Alice      695
Bob        338
Charlie    490
Dan        374
Edith      400
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Oliver', 'amount': 344, 'transaction-id': 123},
 {'id': 0, 'name': 'Oliver', 'amount': 355, 'transaction-id': 179},
 {'id': 0, 'name': 'Oliver', 'amount': 310, 'transaction-id': 317})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Oliver,344,123
1,0,Oliver,355,179
2,0,Oliver,310,317
3,0,Oliver,335,412
4,0,Oliver,340,704


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 193 ms, sys: 9.68 ms, total: 203 ms
Wall time: 1.08 s


name
Alice       21291
Bob         18458
Charlie     21486
Dan         12713
Edith       26156
Frank       16047
George      19717
Hannah      10189
Ingrid      14918
Jerry        6662
Kevin       23077
Laura       29886
Michael     34388
Norbert     18314
Oliver      12062
Patricia    21141
Quinn       16455
Ray         20560
Sarah        7392
Tim          6740
Ursula      23389
Victor      18795
Wendy       35331
Xavier      19208
Yvonne      26163
Zelda       19462
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()